# Transformation Operation Demo

The `transform` operation (along with its variations like `transformWith`) allows arbitrary RDD-to-RDD functions to be applied on a DStream. It can be used to apply any RDD operation that is not exposed in the DStream API. For example, the functionality of joining every batch in a data stream with another dataset is not directly exposed in the DStream API. However, you can easily use `transform to do` this. This enables very powerful possibilities. For example, one can do real-time data cleaning by joining the input data stream with precomputed spam information (maybe generated with Spark as well) and then filtering based on it.
```python
spamInfoRDD = sc.pickleFile(...)  # RDD containing spam information

# join data stream with spam information to do data cleaning
cleanedDStream = wordCounts.transform(lambda rdd: rdd.join(spamInfoRDD).filter(...))
```
Note that the supplied function gets called in every batch interval. This allows you to do time-varying RDD operations, that is, RDD operations, number of partitions, broadcast variables, etc. can be changed between batches.


### Demo
Create DStream from text file

In [1]:
import findspark
# TODO: your path will likely not have 'jilg' in it. Change it to reflect your path.
findspark.init('/home/jilg/Downloads/spark-3.3.2-bin-hadoop3')

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
import sys, re
import random
from apache_log_parser import ApacheAccessLog

In [5]:
random.seed(15)
conf = (SparkConf().setMaster("local[2]").setAppName("TextUpdater").set("spark.executor.memory", "2g"))

#sc = SparkContext(conf=conf)
sc = SparkContext.getOrCreate()

ssc = StreamingContext(sc, 1)
ssc.checkpoint("checkpoint")

In [6]:
# TODO: Change this directory to match yours
directory = 'file:///home/jilg/python-spark-streaming/2_basics/logs'

In [7]:
log_data = ssc.textFileStream(directory)
access_log_dstream = log_data.map(ApacheAccessLog.parse_from_log_line).filter(lambda parsed_line: parsed_line is not None)
access_log_dstream.pprint(num = 30)

In [8]:
# Implementing a transformed stream here using the `transform()` function
def mapIPValues(rdd):
    # Dummy Implementation
    rdd1 = rdd.map(lambda parsed_line: (parsed_line.ip, 1))
    return rdd1

In [9]:
transformed_access_log_dstream = access_log_dstream.transform(mapIPValues)
transformed_access_log_dstream.pprint(num = 30)

Note: the spark streaming checks for any updates to this directory. First, start this program, and then copy the log file logs/access_log.log to 'directory' location

In [10]:
ssc.start() 
#ssc.awaitTermination()

-------------------------------------------
Time: 2023-02-19 09:24:35
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 09:24:35
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 09:24:36
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 09:24:36
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 09:24:37
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 09:24:37
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 09:24:38
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 09:24:38
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 09:24:39
----------

In [11]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2023-02-19 09:24:51
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 09:24:51
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 09:24:52
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 09:24:52
-------------------------------------------



## References
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#transform-operation
